In [13]:
# 导入所需文件
import numpy as np
from sklearn.model_selection import ShuffleSplit
from data_utils import ENTITIES, Documents, Dataset, SentenceExtractor, make_predictions
from data_utils import Evaluator
from gensim.models import Word2Vec

In [14]:
# 数据文件读取
data_dir = "./data/train"
ent2idx = dict(zip(ENTITIES, range(1, len(ENTITIES) + 1)))
idx2ent = dict([(v, k) for k, v in ent2idx.items()])

In [15]:
# 训练集，测试集切分与打乱
docs = Documents(data_dir=data_dir)
rs = ShuffleSplit(n_splits=1, test_size=20, random_state=2018)
train_doc_ids, test_doc_ids = next(rs.split(docs))
train_docs, test_docs = docs[train_doc_ids], docs[test_doc_ids]

In [16]:
# 模型参数赋值
num_cates = max(ent2idx.values()) + 1
sent_len = 64
vocab_size = 3000
emb_size = 100
sent_pad = 10
sent_extrator = SentenceExtractor(window_size=sent_len, pad_size=sent_pad)
train_sents = sent_extrator(train_docs)
test_sents = sent_extrator(test_docs)

train_data = Dataset(train_sents, cate2idx=ent2idx)
train_data.build_vocab_dict(vocab_size=vocab_size)

test_data = Dataset(test_sents, word2idx=train_data.word2idx, cate2idx=ent2idx)
vocab_size = len(train_data.word2idx)

In [17]:
# 构建词嵌入模型
w2v_train_sents = []
for doc in docs:
    w2v_train_sents.append(list(doc.text))
w2v_model = Word2Vec(w2v_train_sents, vector_size=emb_size)

w2v_embeddings = np.zeros((vocab_size, emb_size))
for char, char_idx in train_data.word2idx.items():
    if char in w2v_model.wv:
        w2v_embeddings[char_idx] = w2v_model.wv[char]

In [18]:
# 构建RNN模型加crf模型
import keras
from keras.layers import Input, SimpleRNN, Embedding, Bidirectional
from keras_contrib.layers import CRF
from keras.models import Model



def build_rnn_model(num_cates, seq_len, vocab_size, model_opts=dict()):
    opts = {
        'emb_size': 256,
        'emb_trainable': True,
        'emb_matrix': None,
        'rnn_units': 256,
        'optimizer': keras.optimizers.Adam()
    }
    opts.update(model_opts)

    input_seq = Input(shape=(seq_len,), dtype='int32')
    if opts.get('emb_matrix') is not None:
        embedding = Embedding(vocab_size, opts['emb_size'], 
                              weights=[opts['emb_matrix']],
                              trainable=opts['emb_trainable'])
    else:
        embedding = Embedding(vocab_size, opts['emb_size'])
    x = embedding(input_seq)
    rnn = SimpleRNN(opts['rnn_units'], return_sequences=True)
    x = rnn(x)
    crf = CRF(num_cates, sparse_target=True)
    output = crf(x)

    model = Model(input_seq, output)
    model.compile(opts['optimizer'], loss=crf.loss_function, metrics=[crf.accuracy])
    return model

Using TensorFlow backend.


In [19]:
# 循环神经网络+CRF条件随机场实例化
seq_len = sent_len + 2 * sent_pad
model = build_rnn_model(num_cates, seq_len=seq_len, vocab_size=vocab_size,model_opts={'emb_matrix': w2v_embeddings, 'emb_size': 100, 'emb_trainable': False})
model.summary()

c:\Users\33398\.conda\envs\Aliyun\lib\site-packages\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
c:\Users\33398\.conda\envs\Aliyun\lib\site-packages\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 84)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 84, 100)           215400    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 84, 256)           91392     
_________________________________________________________________
crf_1 (CRF)                  (None, 84, 16)            4400      
Total params: 311,192
Trainable params: 95,792
Non-trainable params: 215,400
_________________________________________________________________


In [20]:
# 训练集，测试集形状
train_X, train_y = train_data[:]
print('train_X.shape', train_X.shape)
print('train_y.shape', train_y.shape)

train_X.shape (2622, 84)
train_y.shape (2622, 84, 1)


In [21]:
# 循环神经网络与条件随机场模型训练
model.fit(train_X, train_y, batch_size=64, epochs=10)

Epoch 1/10
2622/2622 [==============================] - 6s 2ms/step - loss: 1.0604 - crf_viterbi_accuracy: 0.7163
Epoch 2/10
2622/2622 [==============================] - 5s 2ms/step - loss: 0.6782 - crf_viterbi_accuracy: 0.8049
Epoch 3/10
2622/2622 [==============================] - 5s 2ms/step - loss: 0.6053 - crf_viterbi_accuracy: 0.8208
Epoch 4/10
2622/2622 [==============================] - 5s 2ms/step - loss: 0.5646 - crf_viterbi_accuracy: 0.8295
Epoch 5/10
2622/2622 [==============================] - 5s 2ms/step - loss: 0.5304 - crf_viterbi_accuracy: 0.8367
Epoch 6/10
2622/2622 [==============================] - 5s 2ms/step - loss: 0.4983 - crf_viterbi_accuracy: 0.8432
Epoch 7/10
2622/2622 [==============================] - 5s 2ms/step - loss: 0.4719 - crf_viterbi_accuracy: 0.8494
Epoch 8/10
2622/2622 [==============================] - 5s 2ms/step - loss: 0.4547 - crf_viterbi_accuracy: 0.8524
Epoch 9/10
2622/2622 [==============================] - 5s 2ms/step - loss: 0.4363 - crf

In [22]:
# 模型预测
test_X, _ = test_data[:]
preds = model.predict(test_X, batch_size=64, verbose=True)
pred_docs = make_predictions(preds, test_data, sent_pad, docs, idx2ent)

2644/2644 [==============================] - 2s 734us/step


In [23]:
# 输出评价指标
f_score, precision, recall = Evaluator.f1_score(test_docs, pred_docs)
print('f_score: ', f_score)
print('precision: ', precision)
print('recall: ', recall)

f_score:  0.6710640346653828
precision:  0.6514910722632514
recall:  0.6918494986597836


In [24]:
# 测试样本展示
sample_doc_id = list(pred_docs.keys())[3]
# test_docs[sample_doc_id]